I used this log file ==> https://www.kaggle.com/datasets/eliasdabbas/web-server-access-logs

In [ ]:
import psycopg2
import re
from datetime import datetime

In [ ]:
def check_exist_DataBase(db_name):
    conn = psycopg2.connect(
        user="postgres",
        password="YOUR postgreSQL PASSWORD",
        host="localhost",
        port="5432",
        database="postgres"    )
    
#     Disable autocommit mode which disables the transaction block. 
#     This allows us to execute the CREATE DATABASE statement successfully.
    conn.autocommit = True

    cursor = conn.cursor()
    # Define the name of the database you want to check
    database_name = db_name
    # Check if the database already exists
    cursor.execute("SELECT 1 FROM pg_catalog.pg_database WHERE datname = %s", (database_name,))
    database_exists = cursor.fetchone()

    # If the database does not exist, create it
    if not database_exists:
        cursor.execute(f"CREATE DATABASE {database_name}")
        conn.close()
        
        conn = psycopg2.connect(
            user="postgres",
            password="mh",
            host="localhost",
            port="5432",
            database=database_name )

        cursor = conn.cursor()

        # Create a table within the database
        cursor.execute("""
            CREATE TABLE IF NOT EXISTS logs (
                ip_address VARCHAR(16),
                timestamp VARCHAR(19),
                request TEXT,
                status_code INT,
                response_size INT,
                referrer TEXT,
                user_agent TEXT )    """)

        print(f"The database '{database_name}' and Table Logs has been created.")
    else:
        print(f"The database '{database_name}' already exists.")

    # Commit the changes and close the connection
    conn.commit()
    cursor.close()
    conn.close()

In [ ]:
def extract_data(log_line):
    # Define regular expressions to extract relevant information
    pattern = r'^(\S+) - - \[(.*?)\] "(\S+ \S+ \S+)" (\d+) (\d+) "(.*?)" "(.*?)"'
    match = re.match(pattern, log_line)
    
    if match:
        # Extract individual fields from the log line
        ip_address = match.group(1)
        timestamp = datetime.strptime(match.group(2), "%d/%b/%Y:%H:%M:%S %z")
        request = match.group(3)
        status_code = int(match.group(4))
        response_size = int(match.group(5))
        referrer = match.group(6)
        user_agent = match.group(7)
        
        # Return the extracted data as a dictionary
        return {
            "ip_address": ip_address,
            "timestamp": timestamp,
            "request": request,
            "status_code": status_code,
            "response_size": response_size,
            "referrer": referrer,
            "user_agent": user_agent
        }
    else:
        return None

def transform_data(data):
    # Perform any necessary transformations on the extracted data
    # In this example, we can convert the timestamp to a string format
    data["timestamp"] = data["timestamp"].strftime("%Y-%m-%d %H:%M:%S")
    
    return data

def load_data(n, data):
    # Implement the data loading process
    # In this example, we'll simply print the transformed data
    insert(n, data["ip_address"], data["timestamp"], data["request"], data["status_code"], data["response_size"], data["referrer"], data["user_agent"])

In [ ]:
def insert(n, ip_address, timestamp, request, status_code, response_size, referrer, user_agent):
    try:
        connection = psycopg2.connect(
            user="postgres",
            password="mh",
            host="localhost",
            port="5432",
            database="web_server_log"
        )

        cursor = connection.cursor()
        pg_insert = """INSERT INTO public."logs" 
            ("ip_address", "timestamp", "request", "status_code", "response_size", "referrer", "user_agent")
            VALUES (%s, %s, %s, %s, %s, %s, %s)"""
        
        Inserted_Values = (ip_address, timestamp, request, status_code, response_size, referrer, user_agent )
            

        cursor.execute(pg_insert, Inserted_Values)
        connection.commit()

        print(n, "th Record Successfully inserted ")
    except(Exception, psycopg2.Error) as error:
        print("Error connecting to PosgreSQL database", error)
        connection = None

    finally:
        if(connection):
            cursor.close()
            connection.close()

In [ ]:
check_exist_DataBase("web_server_log")
n = 1

with open(r'Datasets\access.log', 'r') as file:
    for line in file:

        # Extract data from the log line
        extracted_data = extract_data(line)

        # Transform the extracted data
        transformed_data = transform_data(extracted_data)

        # Load the transformed data
        load_data(n, transformed_data)
        n += 1        